In [2]:
from ultralytics import YOLO
import os

# Get the absolute path to your dataset YAML file
DATASET_YAML = "/Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset.yaml"

# Initialize the model
model = YOLO("yolov5n.pt")

# Train the model using your custom dataset
results = model.train(
    data=DATASET_YAML,  # Path to your dataset YAML file
    epochs=100,         # Number of training epochs
    imgsz=640,         # Image size
    batch=16,          # Batch size
    name='yolov5_custom'  # Name for the experiment
)

PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Ultralytics 8.3.40 🚀 Python-3.12.3 torch-2.5.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov5n.pt, data=/Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov5_custom3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=Fal

train: Scanning /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset/train... 0 images, 11259 backgrounds, 0 corrupt: 100%|██████████| 11259/11259 [00:02<00:00, 5281.22it/s]

train: WARNING ⚠️ No labels found in /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset/train.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


train: New cache created: /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset/train.cache
WARNING ⚠️ No labels found in /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


val: Scanning /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset/valid... 0 images, 1879 backgrounds, 0 corrupt: 100%|██████████| 1879/1879 [00:00<00:00, 5557.32it/s]

val: WARNING ⚠️ No labels found in /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset/valid.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset/valid.cache
WARNING ⚠️ No labels found in /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/dataset/valid.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/runs/detect/yolov5_custom3/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/matt/Documents/School/CPSC393/MachineLearningFinalProject/runs/detect/yolov5_custom3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G          0      127.2          0          0        640:   1%|          | 5/704 [00:33<1:18:33,  6.74s/it]


KeyboardInterrupt: 

In [ ]:
import os
import random
from pathlib import Path

# Create output directory if it doesn't exist
output_dir = "inference_output"
os.makedirs(output_dir, exist_ok=True)

# Get a random image from your test dataset
test_dir = os.path.join(DATASET_PATH, "test")
test_images = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
random_image = os.path.join(test_dir, random.choice(test_images))

# Run inference
results = model.predict(
    source=random_image,
    save=True,              # Save results
    project=output_dir,     # Output directory
    name='random_test',     # Name of the run
    conf=0.25,             # Confidence threshold
    save_txt=True          # Save labels
)

print(f"Processed image: {random_image}")
print(f"Results saved to: {output_dir}/random_test")